In [1]:
import pandas as pd
df = pd.read_csv('/kaggle/input/imdb-dataset-of-50k-movie-reviews/IMDB Dataset.csv')
df


,review,sentiment
0,One of the other reviewers has mentioned that ...,positive
1,A wonderful little production. <br /><br />The...,positive
2,I thought this was a wonderful way to spend ti...,positive
3,Basically there's a family where a little boy ...,negative
4,"Petter Mattei's ""Love in the Time of Money"" is...",positive
...,...,...
49995,I thought this movie did a down right good job...,positive
49996,"Bad plot, bad dialogue, bad acting, idiotic di...",negative
49997,I am a Catholic taught in parochial elementary...,negative
49998,I'm going to have to disagree with the previou...,negative


In [2]:
import re


def preproc(x):
    x = re.sub(r"<br\s*/?>", " ", x)  
    x = re.sub(r"[^a-zA-Z0-9\s]", "", x) 
    x = x.lower()  
    return x


df["review"] = df["review"].apply(preproc)
df["sentiment"] = df["sentiment"].map({"positive": 1, "negative": 0})
df

,review,sentiment
0,one of the other reviewers has mentioned that ...,1
1,a wonderful little production the filming te...,1
2,i thought this was a wonderful way to spend ti...,1
3,basically theres a family where a little boy j...,0
4,petter matteis love in the time of money is a ...,1
...,...,...
49995,i thought this movie did a down right good job...,1
49996,bad plot bad dialogue bad acting idiotic direc...,0
49997,i am a catholic taught in parochial elementary...,0
49998,im going to have to disagree with the previous...,0


In [3]:
corpus = " ".join(df["review"].tolist())

In [4]:
from collections import Counter

vocab_size = 20000
word_freq = Counter(corpus.split())
most_freq = sorted(word_freq.items(), key=lambda x: x[1], reverse=True)
most_freq_words = [word for word, freq in most_freq[:vocab_size]]
vocab = {word: i + 1 for i, word in enumerate(most_freq_words)}

In [5]:
import json
with open("vocab.json", "w") as f:
    json.dump(vocab, f)

In [6]:
def encode(text):
    return [vocab.get(word, 0) for word in text.split()]

df["review"] = df["review"].apply(encode)
df

,review,sentiment
0,"[27, 4, 1, 75, 1944, 44, 1062, 11, 100, 143, 3...",1
1,"[3, 384, 116, 356, 1, 1338, 2937, 6, 51, 17935...",1
2,"[9, 192, 10, 12, 3, 384, 94, 5, 1120, 57, 19, ...",1
3,"[667, 214, 3, 233, 113, 3, 116, 435, 3556, 120...",0
4,"[0, 0, 109, 7, 1, 57, 4, 285, 6, 3, 2077, 1375...",1
...,...,...
49995,"[9, 192, 10, 16, 112, 3, 181, 200, 48, 286, 8,...",1
49996,"[77, 111, 77, 402, 77, 110, 2997, 963, 1, 626,...",0
49997,"[9, 227, 3, 3510, 4325, 7, 0, 8342, 4602, 31, ...",0
49998,"[140, 161, 5, 24, 5, 2964, 14, 1, 899, 912, 2,...",0


In [7]:
max_len = 200  
new_data = []

for i, row in df.iterrows():
    full_review = row["review"]
    sentiment = row["sentiment"]
    chunks = [full_review[j : j + max_len] for j in range(0, len(full_review), max_len)]

    for chunk in chunks:
        new_data.append([chunk, sentiment])

df = pd.DataFrame(new_data, columns=["review", "sentiment"])

In [8]:
max_len = 200
def pad(text):
    return text + [20001] * (max_len - len(text))

df["review"] = df["review"].apply(pad)
df["review"].apply(len)

0        200
1        200
2        200
3        200
4        200
        ... 
79534    200
79535    200
79536    200
79537    200
79538    200
Name: review, Length: 79539, dtype: int64

In [9]:
df.to_csv("data.csv", index=False)

In [10]:
import torch.nn as nn
import torch


class RNNModel(nn.Module):
    def __init__(self, embedding_dim, hidden_dim, vocab_size):
        super(RNNModel, self).__init__()
        self.embedding = nn.Embedding(vocab_size, embedding_dim, padding_idx=20001)
        self.rnn = nn.RNN(embedding_dim, hidden_dim, batch_first=True)
        self.fc = nn.Linear(hidden_dim, 1)
        self.hidden_dim = hidden_dim

    def forward(self, x, h=None):
        x = self.embedding(x)

        if h is None:
            h = torch.zeros(1, x.size(0), self.hidden_dim).to("cuda")
        x, h = self.rnn(x, h)
        x = x[:, -1, :]
        x = self.fc(x)
        return x, h.detach()

In [11]:
import torch.nn as nn
import torch


class LSTMModel(nn.Module):
    def __init__(self, embedding_dim, hidden_dim, vocab_size):
        super(LSTMModel, self).__init__()
        self.embedding = nn.Embedding(vocab_size, embedding_dim, padding_idx=20001)
        self.lstm = nn.LSTM(embedding_dim, hidden_dim, batch_first=True)
        self.fc = nn.Linear(hidden_dim, 1)
        self.hidden_dim = hidden_dim

    def forward(self, x, h=None):
        x = self.embedding(x)

        if h is None:
            h = (
                torch.zeros(1, x.size(0), self.hidden_dim).to("cuda"),
                torch.zeros(1, x.size(0), self.hidden_dim).to("cuda"),
            )
        x, h = self.lstm(x, h)
        x = x[:, -1, :]
        x = self.fc(x)
        return x, (h[0].detach(), h[1].detach())

In [12]:
import torch
import torch.nn as nn

class GRUModel(nn.Module):
    def __init__(self, embedding_dim, hidden_dim, vocab_size):
        super(GRUModel, self).__init__()
        self.embedding = nn.Embedding(vocab_size, embedding_dim, padding_idx=20001)
        self.gru = nn.GRU(embedding_dim, hidden_dim, batch_first=True)
        self.fc = nn.Linear(hidden_dim, 1)
        self.hidden_dim = hidden_dim

    def forward(self, x, h=None):
        x = self.embedding(x)
        if h is None:
            h = torch.zeros(1, x.size(0), self.hidden_dim).to("cuda")
        x, h = self.gru(x, h)
        x = x[:, -1, :]
        x = self.fc(x)
        return x, h.detach()

In [13]:
import torch
import torch.nn as nn

class BiLSTMModel(nn.Module):
    def __init__(self, embedding_dim, hidden_dim, vocab_size):
        super(BiLSTMModel, self).__init__()
        self.embedding = nn.Embedding(vocab_size, embedding_dim, padding_idx=20001)
        self.bilstm = nn.LSTM(embedding_dim, hidden_dim, batch_first=True, bidirectional=True)
        self.fc = nn.Linear(hidden_dim * 2, 1)

    def forward(self, x, h=None):
        x = self.embedding(x)
        if h is None:
            h = (
                torch.zeros(2, x.size(0), self.bilstm.hidden_size).to("cuda"),
                torch.zeros(2, x.size(0), self.bilstm.hidden_size).to("cuda"),
            )
        x, h = self.bilstm(x, h)
        x = x[:, -1, :]
        x = self.fc(x)
        return x, (h[0].detach(), h[1].detach())

In [14]:
import torch
import torch.nn as nn
import torch
import torch.nn as nn

class StackedLSTMModel(nn.Module):
    def __init__(self, embedding_dim, hidden_dim1, hidden_dim2, vocab_size):
        super(StackedLSTMModel, self).__init__()
        self.embedding = nn.Embedding(vocab_size, embedding_dim, padding_idx=20001)
        self.lstm1 = nn.LSTM(embedding_dim, hidden_dim1, batch_first=True)
        self.lstm2 = nn.LSTM(hidden_dim1, hidden_dim2, batch_first=True)
        self.fc = nn.Linear(hidden_dim2, 1)  
        self.hidden_dim1 = hidden_dim1
        self.hidden_dim2 = hidden_dim2

    def forward(self, x, h=None):
        x = self.embedding(x)
        if h is None:
            h1 = (
                torch.zeros(1, x.size(0), self.hidden_dim1, device=x.device),
                torch.zeros(1, x.size(0), self.hidden_dim1, device=x.device),
            )
            h2 = (
                torch.zeros(1, x.size(0), self.hidden_dim2, device=x.device),
                torch.zeros(1, x.size(0), self.hidden_dim2, device=x.device),
            )
        else:
            h1 = h[:2]
            h2 = h[2:]
        x, h1 = self.lstm1(x, h1)
        x, h2 = self.lstm2(x, h2)
        x = x[:, -1, :]
        x = self.fc(x)
        return x, (h1[0].detach(), h1[1].detach(), h2[0].detach(), h2[1].detach())

In [15]:
import torch
import torch.nn as nn

class StackedBiLSTMModel(nn.Module):
    def __init__(self, embedding_dim, hidden_dim1, hidden_dim2, vocab_size):
        super(StackedBiLSTMModel, self).__init__()
        self.embedding = nn.Embedding(vocab_size, embedding_dim, padding_idx=20001)
        self.bilstm1 = nn.LSTM(embedding_dim, hidden_dim1, batch_first=True, bidirectional=True)
        self.bilstm2 = nn.LSTM(hidden_dim1 * 2, hidden_dim2, batch_first=True, bidirectional=True)
        self.fc = nn.Linear(hidden_dim2 * 2, 1)

    def forward(self, x, h=None):
        x = self.embedding(x)
        if h is None:
            h1 = (
                torch.zeros(2, x.size(0), self.bilstm1.hidden_size, device=x.device),
                torch.zeros(2, x.size(0), self.bilstm1.hidden_size, device=x.device),
            )
            h2 = (
                torch.zeros(2, x.size(0), self.bilstm2.hidden_size, device=x.device),
                torch.zeros(2, x.size(0), self.bilstm2.hidden_size, device=x.device),
            )
        else:
            h1 = h[:2] 
            h2 = h[2:]  
        x, h1 = self.bilstm1(x, h1)
        x, h2 = self.bilstm2(x, h2)
        x = x[:, -1, :]
        x = self.fc(x)
        return x, (h1[0].detach(), h1[1].detach(), h2[0].detach(), h2[1].detach())

In [16]:
import torch.optim as optim

batch_size = 256
seq_len = 200
learning_rate = 0.001
vocab_size = 20002
hidden_dim = 128

criterion = nn.BCEWithLogitsLoss()

In [17]:
from torch.utils.data import DataLoader, TensorDataset
train = TensorDataset(
    torch.tensor(df["review"][:50000].tolist(), dtype=torch.long),
    torch.tensor(df["sentiment"][:50000].tolist(), dtype=torch.float32),
)
test = TensorDataset(
    torch.tensor(df["review"][50000:].tolist(), dtype=torch.long),
    torch.tensor(df["sentiment"][50000:].tolist(), dtype=torch.float32),
)
train_loader = DataLoader(train, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(test, batch_size=batch_size, shuffle=False)

In [18]:
from tqdm import tqdm
from torch.utils.tensorboard import SummaryWriter
import torch
import torch.nn.functional as F
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix


def train_loop(model, bidir, epochs, model_name):
    model = model.to("cuda")
    writer = SummaryWriter(log_dir=f"runs/{model_name}")

    for epoch in range(epochs):
        hidden_state = None
        model.train()
        train_tqdm = tqdm(train_loader, desc=f"Epoch {epoch+1}/{epochs} [Train]")
        
        total_train_loss = 0
        train_preds, train_labels_list = [], []
    
        for i, (train_data, train_labels) in enumerate(train_tqdm):
            train_data, train_labels = train_data.to("cuda"), train_labels.to("cuda")
            if i == len(train_loader) - 1:  
                hidden_state = None
                
            optimizer.zero_grad()
            output, hidden_state = model(train_data, hidden_state)
            output = output.squeeze()
            
            if bidir:
                hidden_state = tuple(h.detach() for h in hidden_state)
            else:
                hidden_state = hidden_state.detach()
            
            loss = criterion(output, train_labels)
            loss.backward()
            optimizer.step()
    
            preds = (torch.sigmoid(output) > 0.5).cpu().numpy()
            train_preds.extend(preds)
            train_labels_list.extend(train_labels.cpu().numpy())

            total_train_loss += loss.item()
            train_tqdm.set_postfix(loss=loss.item())
    
        avg_train_loss = total_train_loss / len(train_loader)
        train_acc = accuracy_score(train_labels_list, train_preds)

        writer.add_scalar(f"{model_name}/trainLoss", avg_train_loss, epoch+1)
        writer.add_scalar(f"{model_name}/trainAcc", train_acc, epoch+1)

        # ------------------- Validation Loop -------------------
        model.eval()
        val_tqdm = tqdm(test_loader, desc=f"Epoch {epoch+1}/{epochs} [Val]")
    
        total_val_loss = 0
        val_preds, val_labels_list = [], []
        hidden_state = None
        with torch.no_grad():
            for i, (val_data, val_labels) in enumerate(val_tqdm):
                val_data, val_labels = val_data.to("cuda"), val_labels.to("cuda")
                if i == len(test_loader) - 1:
                    hidden_state = None
                
                output, hidden_state = model(val_data, hidden_state)
                output = output.squeeze()
                
                if bidir:
                    hidden_state = tuple(h.detach() for h in hidden_state)
                else:
                    hidden_state = hidden_state.detach()
                
                loss = criterion(output, val_labels)
                total_val_loss += loss.item()
    
                preds = (torch.sigmoid(output) > 0.5).cpu().numpy()
                val_preds.extend(preds)
                val_labels_list.extend(val_labels.cpu().numpy())

                val_tqdm.set_postfix(loss=loss.item())
    
        avg_val_loss = total_val_loss / len(test_loader)
        val_acc = accuracy_score(val_labels_list, val_preds)

        writer.add_scalar(f"{model_name}/valLoss", avg_val_loss, epoch+1)
        writer.add_scalar(f"{model_name}/valAcc", val_acc, epoch+1)
    
        print(
            f"Epoch {epoch+1}/{epochs} | Train Loss: {avg_train_loss:.4f} | Train Acc: {train_acc:.4f} | Val Loss: {avg_val_loss:.4f} | Val Acc: {val_acc:.4f}"
        )

    writer.close()  

    # ------------------- Final Metrics -------------------
    precision = precision_score(val_labels_list, val_preds)
    recall = recall_score(val_labels_list, val_preds)
    f1 = f1_score(val_labels_list, val_preds)
    conf_matrix = confusion_matrix(val_labels_list, val_preds)

    print("\nFinal Validation Metrics:")
    print(f"Accuracy:  {val_acc:.4f}")
    print(f"Precision: {precision:.4f}")
    print(f"Recall:    {recall:.4f}")
    print(f"F1-score:  {f1:.4f}")
    print(conf_matrix)

In [19]:
model = RNNModel(64, 128, 20002)
optimizer = optim.Adam(model.parameters(), lr=learning_rate)
train_loop(model, False, 50, "RNN")

Epoch 1/50 [Val]: 100%|██████████| 116/116 [00:00<00:00, 219.02it/s, loss=0.71]


Epoch 1/50 | Train Loss: 0.6951 | Train Acc: 0.5025 | Val Loss: 0.6931 | Val Acc: 0.5073


Epoch 2/50 [Val]: 100%|██████████| 116/116 [00:00<00:00, 228.60it/s, loss=0.716]


Epoch 2/50 | Train Loss: 0.6907 | Train Acc: 0.5213 | Val Loss: 0.6945 | Val Acc: 0.5061


Epoch 3/50 [Val]: 100%|██████████| 116/116 [00:00<00:00, 225.33it/s, loss=0.697]


...

Epoch 48/50 | Train Loss: 0.6238 | Train Acc: 0.5995 | Val Loss: 0.7399 | Val Acc: 0.5186


Epoch 49/50 [Val]: 100%|██████████| 116/116 [00:00<00:00, 194.06it/s, loss=0.76]


Epoch 49/50 | Train Loss: 0.6172 | Train Acc: 0.5993 | Val Loss: 0.7486 | Val Acc: 0.5231


Epoch 50/50 [Val]: 100%|██████████| 116/116 [00:00<00:00, 219.96it/s, loss=0.797]


Epoch 50/50 | Train Loss: 0.6125 | Train Acc: 0.6022 | Val Loss: 0.7550 | Val Acc: 0.5300

Final Validation Metrics:
Accuracy:  0.5300
Precision: 0.5322
Recall:    0.5568
F1-score:  0.5442
[[7367 7286]
 [6598 8288]]


In [20]:
model = LSTMModel(64, 128, 20002)
optimizer = optim.Adam(model.parameters(), lr=learning_rate)
train_loop(model, True, 50, "LSTM")

Epoch 1/50 [Val]: 100%|██████████| 116/116 [00:01<00:00, 108.27it/s, loss=0.69]


Epoch 1/50 | Train Loss: 0.6921 | Train Acc: 0.5147 | Val Loss: 0.6892 | Val Acc: 0.5273


Epoch 2/50 [Val]: 100%|██████████| 116/116 [00:01<00:00, 107.19it/s, loss=0.653]


Epoch 2/50 | Train Loss: 0.6754 | Train Acc: 0.5753 | Val Loss: 0.6469 | Val Acc: 0.6464


Epoch 3/50 [Val]: 100%|██████████| 116/116 [00:01<00:00, 107.42it/s, loss=0.652]


...

Epoch 48/50 | Train Loss: 0.1843 | Train Acc: 0.9251 | Val Loss: 0.5503 | Val Acc: 0.8106


Epoch 49/50 [Val]: 100%|██████████| 116/116 [00:01<00:00, 102.33it/s, loss=0.465]


Epoch 49/50 | Train Loss: 0.1751 | Train Acc: 0.9301 | Val Loss: 0.5643 | Val Acc: 0.8123


Epoch 50/50 [Val]: 100%|██████████| 116/116 [00:01<00:00, 103.65it/s, loss=0.466]


Epoch 50/50 | Train Loss: 0.1797 | Train Acc: 0.9262 | Val Loss: 0.5828 | Val Acc: 0.8107

Final Validation Metrics:
Accuracy:  0.8107
Precision: 0.8337
Recall:    0.7799
F1-score:  0.8059
[[12338  2315]
 [ 3277 11609]]


In [21]:
model = GRUModel(64, 128, 20002)
optimizer = optim.Adam(model.parameters(), lr=learning_rate)
train_loop(model, False, 50, "GRU")

Epoch 1/50 [Val]: 100%|██████████| 116/116 [00:00<00:00, 139.56it/s, loss=0.701]


Epoch 1/50 | Train Loss: 0.6918 | Train Acc: 0.5117 | Val Loss: 0.6891 | Val Acc: 0.5225


Epoch 2/50 [Val]: 100%|██████████| 116/116 [00:00<00:00, 140.83it/s, loss=0.668]


Epoch 2/50 | Train Loss: 0.6811 | Train Acc: 0.5493 | Val Loss: 0.6855 | Val Acc: 0.5378


Epoch 3/50 [Val]: 100%|██████████| 116/116 [00:00<00:00, 139.34it/s, loss=0.59]


...

Epoch 48/50 | Train Loss: 0.0383 | Train Acc: 0.9828 | Val Loss: 1.1231 | Val Acc: 0.8089


Epoch 49/50 [Val]: 100%|██████████| 116/116 [00:00<00:00, 127.44it/s, loss=0.711]


Epoch 49/50 | Train Loss: 0.0375 | Train Acc: 0.9840 | Val Loss: 1.0561 | Val Acc: 0.8155


Epoch 50/50 [Val]: 100%|██████████| 116/116 [00:00<00:00, 139.81it/s, loss=0.891]


Epoch 50/50 | Train Loss: 0.0438 | Train Acc: 0.9820 | Val Loss: 1.0772 | Val Acc: 0.8075

Final Validation Metrics:
Accuracy:  0.8075
Precision: 0.8214
Recall:    0.7897
F1-score:  0.8053
[[12097  2556]
 [ 3130 11756]]


In [22]:
model = BiLSTMModel(64, 128, 20002)
optimizer = optim.Adam(model.parameters(), lr=learning_rate)
train_loop(model, True, 50, "BiLSTM")

Epoch 1/50 [Val]: 100%|██████████| 116/116 [00:01<00:00, 61.05it/s, loss=0.715]


Epoch 1/50 | Train Loss: 0.6943 | Train Acc: 0.5036 | Val Loss: 0.6927 | Val Acc: 0.5145


Epoch 2/50 [Val]: 100%|██████████| 116/116 [00:01<00:00, 60.95it/s, loss=0.7]


Epoch 2/50 | Train Loss: 0.6923 | Train Acc: 0.5122 | Val Loss: 0.6925 | Val Acc: 0.5123


Epoch 3/50 [Val]: 100%|██████████| 116/116 [00:01<00:00, 61.31it/s, loss=0.691]


...

Epoch 48/50 | Train Loss: 0.1991 | Train Acc: 0.9147 | Val Loss: 0.6133 | Val Acc: 0.7982


Epoch 49/50 [Val]: 100%|██████████| 116/116 [00:01<00:00, 60.16it/s, loss=0.463]


Epoch 49/50 | Train Loss: 0.1962 | Train Acc: 0.9183 | Val Loss: 0.7870 | Val Acc: 0.7851


Epoch 50/50 [Val]: 100%|██████████| 116/116 [00:02<00:00, 46.52it/s, loss=0.652]


Epoch 50/50 | Train Loss: 0.1929 | Train Acc: 0.9179 | Val Loss: 0.6283 | Val Acc: 0.8035

Final Validation Metrics:
Accuracy:  0.8035
Precision: 0.8006
Recall:    0.8124
F1-score:  0.8065
[[11641  3012]
 [ 2792 12094]]


In [23]:
model = StackedLSTMModel(64, 128, 64, 20002)
optimizer = optim.Adam(model.parameters(), lr=learning_rate)
train_loop(model, True, 50, "StackedLSTM")

Epoch 1/50 [Val]: 100%|██████████| 116/116 [00:01<00:00, 87.90it/s, loss=0.703]


Epoch 1/50 | Train Loss: 0.6921 | Train Acc: 0.5152 | Val Loss: 0.6900 | Val Acc: 0.5266


Epoch 2/50 [Val]: 100%|██████████| 116/116 [00:01<00:00, 86.90it/s, loss=0.695]


Epoch 2/50 | Train Loss: 0.6881 | Train Acc: 0.5359 | Val Loss: 0.6932 | Val Acc: 0.4960


Epoch 3/50 [Val]: 100%|██████████| 116/116 [00:01<00:00, 87.77it/s, loss=0.702]


...

Epoch 48/50 | Train Loss: 0.0805 | Train Acc: 0.9729 | Val Loss: 0.7415 | Val Acc: 0.8166


Epoch 49/50 [Val]: 100%|██████████| 116/116 [00:01<00:00, 86.45it/s, loss=0.579]


Epoch 49/50 | Train Loss: 0.0771 | Train Acc: 0.9738 | Val Loss: 0.7196 | Val Acc: 0.8149


Epoch 50/50 [Val]: 100%|██████████| 116/116 [00:01<00:00, 87.01it/s, loss=0.608]


Epoch 50/50 | Train Loss: 0.0729 | Train Acc: 0.9751 | Val Loss: 0.7327 | Val Acc: 0.8189

Final Validation Metrics:
Accuracy:  0.8189
Precision: 0.8130
Recall:    0.8321
F1-score:  0.8224
[[11804  2849]
 [ 2500 12386]]


In [24]:
model = StackedBiLSTMModel(64, 128, 64, 20002)
optimizer = optim.Adam(model.parameters(), lr=learning_rate)
train_loop(model, True, 50, "StackedBiLSTM")

Epoch 1/50 [Val]: 100%|██████████| 116/116 [00:02<00:00, 44.83it/s, loss=0.637]


Epoch 1/50 | Train Loss: 0.6917 | Train Acc: 0.5217 | Val Loss: 0.6997 | Val Acc: 0.5010


Epoch 2/50 [Val]: 100%|██████████| 116/116 [00:02<00:00, 44.20it/s, loss=0.691]


Epoch 2/50 | Train Loss: 0.6897 | Train Acc: 0.5264 | Val Loss: 0.6928 | Val Acc: 0.5062


Epoch 3/50 [Val]: 100%|██████████| 116/116 [00:02<00:00, 44.64it/s, loss=0.683]


...

Epoch 48/50 | Train Loss: 0.1473 | Train Acc: 0.9477 | Val Loss: 0.5569 | Val Acc: 0.8225


Epoch 49/50 [Val]: 100%|██████████| 116/116 [00:02<00:00, 43.95it/s, loss=0.575]


Epoch 49/50 | Train Loss: 0.1329 | Train Acc: 0.9539 | Val Loss: 0.5596 | Val Acc: 0.8231


Epoch 50/50 [Val]: 100%|██████████| 116/116 [00:02<00:00, 44.01it/s, loss=0.516]


Epoch 50/50 | Train Loss: 0.1229 | Train Acc: 0.9587 | Val Loss: 0.5885 | Val Acc: 0.8234

Final Validation Metrics:
Accuracy:  0.8234
Precision: 0.8404
Recall:    0.8019
F1-score:  0.8207
[[12386  2267]
 [ 2949 11937]]
